# Data Merge
Combining my two data sources into a single, useful DataFrame required a fair amount of work. This notebook puts everything together and converts values for analysis. It is dependent on pickle files saved in notebooks **player_injury_data.ipynb** and **player_nst_data.ipynb**.

### Table of Contents
* [1. Imports and Functions](#sec1)
* [2. Define List of Season Parameters](#sec2)
* [3. Add Season Column to Injuries Dataframe](#sec3)
* [4. Make DataFrame of Games Missed due to Injury by Player and Season](#sec4)
* [5. Change Games Missed Names to Match Stats Names](#sec5)
* [6. Create DataFrame for Missing Game Values for Pre-Stats Seasons](#sec6)
* [7. Merge Stats, Games Missed, and Pre-Stats DataFrames](#sec7)
* [8. Adjust Games Missed if Total Games Exceeds Games in a Season](#sec8)
* [9. Add Time-Related Features](#sec9)
* [10. Add Total Games Played column](#sec10)
* [11. Branch Off a Separate DataFrame for Player Season Averages](#sec11)
* [12. Branch Off a Separate DataFrame for Rolling Averages](#sec12)
* [13. Convert Counting Stats to Per Game Stats](#sec13)
* [14. Save Pickles](#sec14)

<a id='sec1'></a>
### 1. Imports and Functions
* **var_to_pickle**: Writes the given variable to a pickle file
* **read_pickle**: Reads the given pickle file

In [1]:
import datetime
import numpy as np
import pandas as pd
from collections import defaultdict

from nhl_injuries_code import var_to_pickle, read_pickle

%matplotlib inline

<a id='sec2'></a>
### 2. Define List of Season Parameters
Sets the range of seasons I analyzed as well as the number of games per season.

In [2]:
season_years = list(range(2010,2020))
season_month = 9
season_day = 1

# Every season in data range is 82 games except for lockout-shortened 2013 with 48
season_length = defaultdict(lambda:82)
season_length[2013] = 48

<a id='sec3'></a>
### 3. Add Season Column to Injuries Dataframe

In [3]:
injuries_df = read_pickle('../data/injuries_df.pk')
injuries_df['Season'] = 0
min_year = injuries_df['Injury_Date'].min().year
max_year = injuries_df['Injury_Date'].max().year + 1
season = {min_year - 1:datetime.datetime(min_year - 1, season_month, season_day)}
for year in range(min_year, max_year):
    season[year] = datetime.datetime(year, season_month, season_day)
    mask = ((injuries_df['Injury_Date'] > season[year-1])
            & (injuries_df['Injury_Date'] <= season[year]))
    injuries_df.loc[mask, 'Season'] = year

<a id='sec4'></a>
### 4. Make DataFrame of Games Missed due to Injury by Player and Season

In [4]:
missed_df = (injuries_df.groupby(['Name', 'Birth_Date', 'Season'], as_index=False).sum())

# Cap games missed by injury to season length (seasons may be shortened by lockouts)
missed_df.loc[missed_df['Games_Missed'] > season_length[0], 'Games_Missed'] = season_length[0]
for key,val in season_length.items():
    mask = (missed_df['Season'] == key) & (missed_df['Games_Missed'] > val)
    missed_df.loc[mask, 'Games_Missed'] = val

<a id='sec5'></a>
### 5. Change Games Missed Names to Match Stats Names
The inconsistencies are due to the two data sets coming from different sources.

In [5]:
name_changes = [
    ('Alex Burmistrov', 'Alexander Burmistrov'),
    ('Alexander Petrovic', 'Alex Petrovic'),
    ('Alexei Marchenko', 'Alexey Marchenko'),
    ('Matt Benning', 'Matthew Benning'),
    ('Michael Cammalleri', 'Mike Cammalleri'),
    ('Mike Sauer', 'Michael Sauer'),
    ('Mike Zigomanis', 'Michael Zigomanis'),
    ('P.A. Parenteau', 'PA Parenteau'),
    ('T.J. Brodie', 'TJ Brodie'),
    ('T.J. Galiardi', 'TJ Galiardi')
]
for old_name,new_name in name_changes:
    missed_df.loc[missed_df['Name'] == old_name, 'Name'] = new_name

<a id='sec6'></a>
### 6. Create DataFrame for Missing Game Values for Pre-Stats Seasons
The stats DataFrame contains all of the seasons that comprise my data set, but I needed some additional injury data from earlier seasons to properly calculate numbers for the prior injuries columns as many players started their careers before the range of study.

In [6]:
pre_stats = (missed_df[missed_df['Season'] < season_years[0]]
             .sort_values(by=['Name', 'Birth_Date', 'Season'])
             .reset_index(drop=True))
groupby = pre_stats.groupby(['Name', 'Birth_Date'])
last_missed = groupby['Games_Missed'].last()
first_season = groupby['Season'].first()
last_season = groupby['Season'].last()
total_missed = groupby['Games_Missed'].sum()
pre_stats = (pd.concat([total_missed, last_missed, first_season, last_season], axis=1)
               .reset_index())
pre_stats.columns = ['Name', 'Birth_Date', 'Pre_TGM', 'Pre_GM', 'First', 'Last']
pre_stats['Pre_NS'] = season_years[0] - pre_stats['First']
pre_stats.loc[pre_stats['First'] < 2005, 'Pre_NS'] -= 1
pre_stats.loc[pre_stats['Last'] != season_years[0]-1, 'Pre_GM'] = 0
pre_stats.drop(['First', 'Last'], axis=1, inplace=True)

<a id='sec7'></a>
### 7. Merge Stats, Games Missed, and Pre-Stats DataFrames
Due to some birthday inconsistencies, I only used birthdays to merge players that share the same name.

In [7]:
stats_df = read_pickle('../data/stats_df.pk')

# Not all player birthdays are consistent between DataFrames, so only use Birth_Date as
# a merge condition if multiple players share the same name
bdays_per_name = stats_df.groupby('Name')['Birth_Date'].nunique()
multiple_names = bdays_per_name[(bdays_per_name > 1)].index
multiples_df = stats_df[stats_df['Name'].isin(multiple_names)]
multiples_df = multiples_df.merge(missed_df, how='left', on=['Name', 'Birth_Date', 'Season'])
multiples_df = multiples_df.merge(pre_stats, how='left', on=['Name', 'Birth_Date'])

# Recombine multiple name and single name DataFrames
df = stats_df[~stats_df['Name'].isin(multiple_names)]
df = df.merge(missed_df[['Name', 'Season', 'Games_Missed']], how='left', on=['Name', 'Season'])
df = df.merge(pre_stats.drop(['Birth_Date'], axis=1), how='left', on=['Name'])
df = df.append(multiples_df)
df = df.sort_values(by=['Name', 'Birth_Date', 'Season']).reset_index(drop=True)
df.fillna(0, inplace=True)

<a id='sec8'></a>
### 8. Adjust Games Missed if Total Games Exceeds Games in a Season
Total Games equals Games Missed plus Games Played. Some players do actually play more than the regulation number of games due to being traded to a team that has games in hand on their former team. In most situations, total games should less than or equal to the number of games in a season.

In [8]:
max_missed = (season_length[0] - df['Games_Played']).clip(0, season_length[0])
df['Games_Missed'] = df['Games_Missed'].clip(0, max_missed)

<a id='sec9'></a>
### 9. Add Time-Related Features
I added columns for player age by season, games missed last season, and average games missed per season for all previous seasons.

In [9]:
# Age per season
season_starts = df['Season'].apply(lambda x: season[x])
df['Age'] = (season_starts - df['Birth_Date']).dt.days // 365

# Games Missed last season
df['Last_Games_Missed'] = df.groupby(['Name', 'Birth_Date'])['Games_Missed'].shift(1)
df.loc[df['Season'] == season_years[0], 'Last_Games_Missed'] = df['Pre_GM']
df['Last_Games_Missed'] = df['Last_Games_Missed'].fillna(0).astype(int)

# Average Games Missed for all previous seasons
df_temp1 = df[['Name', 'Birth_Date', 'Season']]
df_temp2 = df[['Name', 'Birth_Date', 'Season', 'Games_Missed']]
df_temp1 = pd.merge(df_temp1, df_temp2, on=['Name', 'Birth_Date'])
df_temp1 = df_temp1[df_temp1['Season_x'] > df_temp1['Season_y']]
groupby = df_temp1.groupby(['Name', 'Birth_Date', 'Season_x'])
df_temp1 = groupby[['Games_Missed']].sum()
df_temp1['NS'] = groupby['Games_Missed'].count()
df_temp1.reset_index(inplace=True)
df_temp1.rename(columns={'Season_x':'Season', 'Games_Missed':'TGM'}, inplace=True)
df = df.merge(df_temp1, how='left', on=['Name', 'Birth_Date', 'Season'])
df['TGM'].fillna(0, inplace=True)
df['NS'].fillna(0, inplace=True)
df['Avg_Games_Missed'] = (df['TGM'] + df['Pre_TGM']) / (df['NS'] + df['Pre_NS'])
df['Avg_Games_Missed'] = df['Avg_Games_Missed'].fillna(0)
df.drop(['Pre_TGM', 'Pre_GM', 'Pre_NS', 'TGM', 'NS'], axis=1, inplace=True)

<a id='sec10'></a>
### 10. Add Total Games Played column
I used Total Games Played to calculate per-game averages for the player totals DataFrame as well as to filter out players who do not play consistently in the NHL or barely overlapped with my time range. Ideally I could utilize minor league injury data, but that is even more difficult to find than NHL injury data.

In [10]:
total_games = df.groupby(['Name', 'Birth_Date'], as_index=False)['Games_Played'].sum()
total_games.rename(columns={'Games_Played':'Total_Games_Played'}, inplace=True)
df = df.merge(total_games, how='left', on=['Name', 'Birth_Date'])

<a id='sec11'></a>
### 11. Branch Off a Separate DataFrame for Player Season Averages
This separate DataFrame attempts to capture a player's 'typical' season for the range of observed seasons. All counting stats are averaged by season instead of by Games Played as in the main DataFrame.

In [11]:
per_game_stats = ['Time_On_Ice', 'Points', 'Shots', 'Penalty_Minutes',
                  'Major_Penalties', 'Penalties_Drawn', 'Hits', 'Hits_Taken',
                  'Shots_Blocked']
groupby = df.groupby(['Name', 'Birth_Date'], as_index=False)
player_df = groupby[per_game_stats].sum()
df_temp1 = groupby[['Games_Missed', 'Age']].mean()
df_temp2 = groupby[['Position', 'Height', 'Weight', 'Nationality', 'European',
                    'Russian', 'Total_Games_Played']].last()
player_df = player_df.merge(df_temp1, how='left', on=['Name', 'Birth_Date'])
player_df = player_df.merge(df_temp2, how='left', on=['Name', 'Birth_Date'])
player_df.reset_index(drop=True, inplace=True)

<a id='sec12'></a>
### 12. Branch Off a Separate DataFrame for Rolling Averages
This DataFrame essentially combines the other two by holding both per-season counting stats as well as a rolling average of counting stats from all previous seasons.

In [12]:
df_temp1 = df[['Name', 'Birth_Date', 'Season']]
df_temp2 = df[['Name', 'Birth_Date', 'Season', 'Games_Played'] + per_game_stats]
df_temp1 = pd.merge(df_temp1, df_temp2, on=['Name', 'Birth_Date'])
df_temp1 = df_temp1[df_temp1['Season_x'] > df_temp1['Season_y']]
groupby = df_temp1.groupby(['Name', 'Birth_Date', 'Season_x'])
df_temp1 = groupby.sum()
df_temp1.drop(['Season_y'], axis=1, inplace=True)
avg_cols = ['Avg_%s' % col for col in df_temp1.columns]
df_temp1.columns = avg_cols
df_temp1['NS'] = groupby['Games_Played'].count()
df_temp1.reset_index(inplace=True)
df_temp1.rename(columns={'Season_x':'Season'}, inplace=True)
rolling_df = df.merge(df_temp1, how='left', on=['Name', 'Birth_Date', 'Season'])
rolling_df['NS'].fillna(0, inplace=True)
for col in avg_cols[1:]:
    rolling_df[col] = rolling_df[col] / rolling_df[avg_cols[0]]
rolling_df[avg_cols[0]] = rolling_df[avg_cols[0]] / rolling_df['NS']
rolling_df.drop(['NS'], axis=1, inplace=True)
rolling_df.fillna(0, inplace=True)

<a id='sec13'></a>
### 13. Convert Counting Stats to Per Game Stats
I normalized some stat values to account for differences in games played. Converted values are Time on Ice, Points, Shots, Penalty Minutes, Major Penalties, Penalties Drawn, Hits, Hits Taken, and Shots Blocked.

In [13]:
for stat in per_game_stats:
    df[stat] = df[stat] / df['Games_Played']
    df[stat] = df[stat].fillna(0)
    player_df[stat] = player_df[stat] / player_df['Total_Games_Played']
    rolling_df[stat] = rolling_df[stat] / rolling_df['Games_Played']
    rolling_df[stat] = rolling_df[stat].fillna(0)

<a id='sec14'></a>
### 14. Save Pickles

In [14]:
df_pickle = '../data/merged_data_df.pk'
var_to_pickle(df, df_pickle)
player_pickle = '../data/merged_by_player_df.pk'
var_to_pickle(player_df, player_pickle)
rolling_pickle = '../data/merged_rolling_df.pk'
var_to_pickle(rolling_df, rolling_pickle)